In [1]:
import copy
import math

import numpy as np
import pop_tools

## load POP grid information
## add 3D transformation of tracer points to Dataset, for nearest tracer point determination

In [2]:
grid_ds = pop_tools.get_grid("POP_gx1v7")

grid_ds["xt"] = np.cos((math.pi / 180.0) * grid_ds["TLONG"]) * np.cos((math.pi / 180.0) * grid_ds["TLAT"])
grid_ds["yt"] = np.sin((math.pi / 180.0) * grid_ds["TLONG"]) * np.cos((math.pi / 180.0) * grid_ds["TLAT"])
grid_ds["zt"] = np.sin((math.pi / 180.0) * grid_ds["TLAT"])

## helper functions

In [3]:
def nearest_t_inds(grid_ds, lat, lon):
    """return dict of indices of tracer point in grid_ds nearest to (lat, lon)"""
    # nearest point maximizes dot prod of 3D cartesian transformation of tracer points with
    # 3D cartesian transformation of (lat, lon)
    # fully generic version should account for land-sea mask
    # we're ignoring that here because the nearest cell point locations of interest is not land

    x = math.cos((math.pi / 180.0) * lon) * math.cos((math.pi / 180.0) * lat)
    y = math.sin((math.pi / 180.0) * lon) * math.cos((math.pi / 180.0) * lat)
    z = math.sin((math.pi / 180.0) * lat)

    dotprod = x * grid_ds["xt"] + y * grid_ds["yt"] + z * grid_ds["zt"]

    return dotprod.argmax(...)

In [4]:
def grid_var_str(grid_ds, varname, inds, **kwargs):
    """print grid_ds[varname] value at inds+inds_offset"""
    inds_loc = copy.deepcopy(inds)
    for key in inds:
        inds_loc[key] += kwargs.get("inds_offset", {}).get(key, 0)
    varname_str = kwargs.get("varname_str", varname)
    scalef = kwargs.get("scalef", 1)
    fmt = kwargs.get("fmt", "0.3f")
    return f"{varname_str}={scalef*grid_ds[varname].isel(inds_loc).values:{fmt}}"

In [5]:
def nearest_grid_info_all_vars(grid_ds, lat, lon):
    """print specific grid info for tracer point nearest to (lat, lon)"""
    inds = nearest_t_inds(grid_ds, lat, lon)
    print("Cell Info")
    print(",".join([
        f"{key}={da.values}" for key, da in inds.items()
    ]))
    print(",".join([
        grid_var_str(grid_ds, "TLAT", inds, varname_str="lat"),
        grid_var_str(grid_ds, "TLONG", inds, varname_str="lon"),
    ]))
    print(grid_var_str(grid_ds, "TAREA", inds, varname_str="area(km^2)", scalef=(1.e-2*1.e-3)**2, fmt="0.3e"))
    print(grid_var_str(grid_ds, "DXT", inds, varname_str="dx(km)", scalef=1.e-2*1.e-3, fmt="0.3e"))
    print(grid_var_str(grid_ds, "DYT", inds, varname_str="dy(km)", scalef=1.e-2*1.e-3, fmt="0.3e"))
    print(grid_var_str(grid_ds, "KMT", inds, fmt="0d"))
    print("Cell North-East Corner Info")
    print(",".join([
        grid_var_str(grid_ds, "ULAT", inds, varname_str="lat"),
        grid_var_str(grid_ds, "ULONG", inds, varname_str="lon"),
    ]))
    print("Cell South-East Corner Info")
    inds_offset={"nlat": -1}
    print(",".join([
        grid_var_str(grid_ds, "ULAT", inds, varname_str="lat", inds_offset=inds_offset),
        grid_var_str(grid_ds, "ULONG", inds, varname_str="lon", inds_offset=inds_offset),
    ]))
    print("Cell South-West Corner Info")
    inds_offset={"nlat": -1, "nlon": -1}
    print(",".join([
        grid_var_str(grid_ds, "ULAT", inds, varname_str="lat", inds_offset=inds_offset),
        grid_var_str(grid_ds, "ULONG", inds, varname_str="lon", inds_offset=inds_offset),
    ]))
    print("Cell North-West Corner Info")
    inds_offset={"nlon": -1}
    print(",".join([
        grid_var_str(grid_ds, "ULAT", inds, varname_str="lat", inds_offset=inds_offset),
        grid_var_str(grid_ds, "ULONG", inds, varname_str="lon", inds_offset=inds_offset),
    ]))


## North Pacific SERIES: 50.14°N, 144.75W°

In [6]:
nearest_grid_info_all_vars(grid_ds, lat=50.14, lon=-144.75)

Cell Info
nlat=310,nlon=224
lat=50.283,lon=215.091
area(km^2)=5.117e+03
dx(km)=8.095e+01
dy(km)=6.320e+01
KMT=55
Cell North-East Corner Info
lat=50.513,lon=215.715
Cell South-East Corner Info
lat=49.951,lon=215.594
Cell South-West Corner Info
lat=50.049,lon=214.465
Cell North-West Corner Info
lat=50.613,lon=214.586


## Equatorial Pacific IronEx-1: 5°S, 90W°

In [7]:
nearest_grid_info_all_vars(grid_ds, lat=-5, lon=-90)

Cell Info
nlat=168,nlon=275
lat=-4.942,lon=269.938
area(km^2)=3.701e+03
dx(km)=1.246e+02
dy(km)=2.970e+01
KMT=53
Cell North-East Corner Info
lat=-4.808,lon=270.500
Cell South-East Corner Info
lat=-5.075,lon=270.500
Cell South-West Corner Info
lat=-5.075,lon=269.375
Cell North-West Corner Info
lat=-4.808,lon=269.375


## Southern Ocean SOFeX-N: 56.23°S, 172W°

In [8]:
nearest_grid_info_all_vars(grid_ds, lat=-56.23, lon=-172)

Cell Info
nlat=43,nlon=202
lat=-56.249,lon=187.813
area(km^2)=4.127e+03
dx(km)=6.949e+01
dy(km)=5.940e+01
KMT=58
Cell North-East Corner Info
lat=-55.981,lon=188.375
Cell South-East Corner Info
lat=-56.515,lon=188.375
Cell South-West Corner Info
lat=-56.515,lon=187.250
Cell North-West Corner Info
lat=-55.981,lon=187.250


## Southern Ocean SOFeX-S: 66.45°S, 171.8W°

In [9]:
nearest_grid_info_all_vars(grid_ds, lat=-66.45, lon=-171.8)

Cell Info
nlat=24,nlon=202
lat=-66.399,lon=187.813
area(km^2)=2.974e+03
dx(km)=5.008e+01
dy(km)=5.940e+01
KMT=50
Cell North-East Corner Info
lat=-66.131,lon=188.375
Cell South-East Corner Info
lat=-66.665,lon=188.375
Cell South-West Corner Info
lat=-66.665,lon=187.250
Cell North-West Corner Info
lat=-66.131,lon=187.250
